In [1]:
from src.data._dataobject import DataObject
from src.data.types._text import Text
from src.data.types._nested import Nested
from src.data.types._list import List, Array

class Address(DataObject):
    city = Text()
    street = Text()

class City(DataObject):
    name = Text(is_id=True, nullable=False)


class User(DataObject):
    __table__ = 'users'

    first_name = Text()
    last_name = Text()
    address = Nested(Address)
    cities = List(City)

u = User(
    first_name='Frodo',
    last_name='Baggins',
    address=Address(
        city='Piacenza',
        street='Via San Sisto'
    ),
    cities=[City(name='Milano'), City(name='Piacenza'), City(name='Kigali')]
)


In [2]:
from src.data.types._enum import Enum

class Connections(DataObject):
    SQL = Enum()
    MONGO = Enum()

In [3]:
Connections.MONGO == 'MONGO'

<class 'abc.ABCMeta'>


TypeError: 